AB Testing


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/AdSmartABdata.csv")

In [4]:
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [9]:
# checking the size of the data
df.shape

(8077, 9)

In [10]:
# checking for missing values
df.isnull().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

1. Which online users belong to the control and exposed groups?

In [15]:
df.groupby("experiment")["auction_id"].count()

experiment
control    4071
exposed    4006
Name: auction_id, dtype: int64

2. How are the users targeted?


3. Could we use the counts of yes and no answers to make a judgement on which experiment is 
performing better? For example if #yes > #no for the exposed group than the control group, 
could we declare that the ad had a significant impact Why or why not?


In [20]:
agg = df.groupby("experiment").agg(
{"yes":"sum"})

In [22]:
agg

,yes
experiment,
control,264
exposed,308


In [24]:
agg = df.groupby("experiment").agg(
{"no":"sum"})

In [26]:
agg

,no
experiment,
control,322
exposed,349


In [ ]:
4. What is the statistical process that generates the data? Which kind of statistical model will you use if you were to simulate the data?
